<a href="https://colab.research.google.com/github/jubair231dd/ML/blob/main/Assignment_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download the file

In [1]:
import requests

# url = "https://www.gutenberg.org/cache/epub/11/pg11.txt"
# with open("wonderland.txt", "wb") as f:
#     f.write(response.content)

url = 'https://www.gutenberg.org/cache/epub/70554/pg70554.txt'
response = requests.get(url)

with open("soilders_song.txt", "wb") as f:
    f.write(response.content)


In [3]:
# load ascii text and covert to lowercase
filename = "soilders_song.txt"
raw_text = open(filename, 'r', encoding='utf-8').read()
raw_text = raw_text.lower()
print(len(raw_text))
 
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
print(chars)
char_to_int = dict((c, i) for i, c in enumerate(chars))
print(char_to_int)
 
# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)
print(chars)

317158
['\n', ' ', '!', '"', '#', '$', '%', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '—', '“', '”', '•', '™', '\ufeff']
{'\n': 0, ' ': 1, '!': 2, '"': 3, '#': 4, '$': 5, '%': 6, "'": 7, '(': 8, ')': 9, '*': 10, ',': 11, '-': 12, '.': 13, '/': 14, '0': 15, '1': 16, '2': 17, '3': 18, '4': 19, '5': 20, '6': 21, '7': 22, '8': 23, '9': 24, ':': 25, ';': 26, '?': 27, '[': 28, ']': 29, 'a': 30, 'b': 31, 'c': 32, 'd': 33, 'e': 34, 'f': 35, 'g': 36, 'h': 37, 'i': 38, 'j': 39, 'k': 40, 'l': 41, 'm': 42, 'n': 43, 'o': 44, 'p': 45, 'q': 46, 'r': 47, 's': 48, 't': 49, 'u': 50, 'v': 51, 'w': 52, 'x': 53, 'y': 54, 'z': 55, '—': 56, '“': 57, '”': 58, '•': 59, '™': 60, '\ufeff': 61}
Total Characters:  317158
Total Vocab:  62
['\n', ' ', '!', '"', '#', '$', '%', "'", '(', ')', '*', ',', '-', '.', '/', '

In [4]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  317058


`LSTM layer is going to be used in the model, thus the input tensor should be of dimension (sample, time steps, features)`

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
 
# reshape X to be [samples, time steps, features]
X = torch.tensor(dataX, dtype=torch.float32).reshape(n_patterns, seq_length, 1)
X = X / float(n_vocab)
y = torch.tensor(dataY)
print(X.shape, y.shape)
print(X[:2])

torch.Size([317058, 100, 1]) torch.Size([317058])
tensor([[[0.9839],
         [0.7903],
         [0.5968],
         [0.5484],
         [0.0161],
         [0.7258],
         [0.7581],
         [0.7097],
         [0.6290],
         [0.5484],
         [0.5161],
         [0.7903],
         [0.0161],
         [0.5806],
         [0.8065],
         [0.7903],
         [0.5484],
         [0.6935],
         [0.5000],
         [0.5484],
         [0.7581],
         [0.5806],
         [0.0161],
         [0.5484],
         [0.5000],
         [0.7097],
         [0.7097],
         [0.6452],
         [0.0161],
         [0.7097],
         [0.5645],
         [0.0161],
         [0.7742],
         [0.6129],
         [0.7581],
         [0.0161],
         [0.7581],
         [0.6129],
         [0.5161],
         [0.5968],
         [0.4839],
         [0.7581],
         [0.5323],
         [0.1129],
         [0.7742],
         [0.0161],
         [0.5806],
         [0.7581],
         [0.4839],
         [0.6935],


In [7]:
import torch.optim as optim
import torch.utils.data as data
 
class CharModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.Sequential(
            nn.LSTM(input_size=1,
                    hidden_size=256,
                    num_layers=2,
                    batch_first=True, 
                    dropout = 0.2) 
        )
        self.linear = nn.Sequential(
            nn.Dropout(0.2),
            nn.Linear(256, n_vocab) 
        )
        '''
        self.lstm = nn.LSTM(input_size=1, hidden_size=256, num_layers=2, batch_first=True, dropout = 0.2)
        self.dropout = nn.Dropout(0.2)
        self.linear = nn.Linear(256, n_vocab)
        '''
    def forward(self, x):
        x, _ = self.lstm(x)
        # take only the last output
        x = x[:, -1, :]
        # produce output
        # x = self.linear(self.dropout(x))
        x = self.linear(x)
        return x

In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [ ]:
n_epochs = 20
batch_size = 64
model = CharModel()
model.to(device)
 
optimizer = optim.Adam(model.parameters())
loss_fn = nn.CrossEntropyLoss(reduction="sum")
loader = data.DataLoader(data.TensorDataset(X, y), shuffle=True, batch_size=batch_size)
 
best_model = None
best_loss = np.inf
train_loss = 0
for epoch in range(n_epochs):
    model.train()
    for X_batch, y_batch in loader:
        y_pred = model(X_batch.to(device))
        loss = loss_fn(y_pred, y_batch.to(device))
        train_loss += loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f"Epoch : {epoch} | Train loss :  {train_loss}")
    # Validation
    model.eval()
    loss = 0
    with torch.no_grad():
        for X_batch, y_batch in loader:
            y_pred = model(X_batch.to(device))
            loss += loss_fn(y_pred, y_batch.to(device))
        if loss < best_loss:
            best_loss = loss
            best_model = model.state_dict()
        print(f"Epoch : {epoch} | Test loss :  {loss}")
 
torch.save([best_model, char_to_int], "single-char.pth")

Epoch : 0 | Train loss :  873182.1875
Epoch : 0 | Test loss :  791339.25
Epoch : 1 | Train loss :  1635927.5
Epoch : 1 | Test loss :  692282.875


In [ ]:
seq_length = 100
start = np.random.randint(0, len(raw_text)-seq_length)
prompt = raw_text[start:start+seq_length]
print(prompt)

in: “où est ma chatte?” which
was the first sentence in her french lesson-book. the mouse gave a
sud


In [ ]:
# Generation using the trained model
best_model, char_to_int = torch.load("single-char.pth")
n_vocab = len(char_to_int)
int_to_char = dict((i, c) for c, i in char_to_int.items())
model.load_state_dict(best_model)
 
# randomly generate a prompt
filename = "wonderland.txt"
seq_length = 100
raw_text = open(filename, 'r', encoding='utf-8').read()
raw_text = raw_text.lower()
start = np.random.randint(0, len(raw_text)-seq_length)
prompt = raw_text[start:start+seq_length]
pattern = [char_to_int[c] for c in prompt]
 
model.eval()
print('Prompt: "%s"' % prompt)
with torch.no_grad():
    for i in range(1000):
        # format input array of int into PyTorch tensor
        x = np.reshape(pattern, (1, len(pattern), 1)) / float(n_vocab)
        x = torch.tensor(x, dtype=torch.float32)
        # generate logits as output from the model
        prediction = model(x.to(device))
        # convert logits into one character
        index = int(prediction.argmax())
        result = int_to_char[index]
        print(result, end="")
        # append the new character into the prompt for the next iteration
        pattern.append(index)
        pattern = pattern[1:]
print()
print("Done.")

Prompt: "ow smaller, i can creep under the door; so either way i’ll
get into the garden, and i don’t care whi"
ne you the white rabbit iad a little better of the sea—”

“i dan’t be anoo iare in the same the same that iise all the sueen of the semeskerg was the march hare and the march hare and the march hare and the march hare and the march hare and the march hare and the march hare and the march hare and the march hare and the march hare and the march hare and the march hare and the march hare and the march hare and the march hare and t